In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

def drop_vars(df):
    tmp=df.shape[1]
    df = df[df.columns[[True]+list((df.var()!=0))]]
    print('0 var:',tmp-df.shape[1])
    
    corr_matrix = df[df.columns[2:]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    tmp=df.shape[1]
    df=df.drop(to_drop, axis=1)
    print('Corr>0.95:',tmp-df.shape[1],'Now:',)
    
    corrs = dict()
    for i in range(df.shape[1]-2):
        corrs[df.columns[2+i]] = np.corrcoef(df['target'],df[df.columns[2+i]])[0,1]
    s = [k for k in corrs if abs(corrs[k])<0.1]
    tmp=df.shape[1]
    df=df.drop(s, axis=1)
    print('Corr Target <0.1:',tmp-df.shape[1])
    
    return df
train = drop_vars(train)

0 var: 256
Corr>0.95: 139 Now:
Corr Target <0.1: 4206


## Test Improvement

In [6]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(np.sqrt(np.mean((y_pred - y_true)**2))) 

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'rmse'},
    'num_leaves': 50,
    'learning_rate': 0.008,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 4,
    'max_depth': -1,
    'reg_alpha': 0.3,
    'reg_lambda': 0.1,
    'min_child_weight': 10,
    'zero_as_missing': True,
    'verbose': 1
}

print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Start predicting...')
# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
print(root_mean_squared_error(y_pred, y_test))


Start training...
[1]	valid_0's rmse: 1.73905
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 1.73622
[3]	valid_0's rmse: 1.7331
[4]	valid_0's rmse: 1.73015
[5]	valid_0's rmse: 1.72709
[6]	valid_0's rmse: 1.72404
[7]	valid_0's rmse: 1.72075
[8]	valid_0's rmse: 1.71815
[9]	valid_0's rmse: 1.71527
[10]	valid_0's rmse: 1.71258
[11]	valid_0's rmse: 1.71013
[12]	valid_0's rmse: 1.70768
[13]	valid_0's rmse: 1.70481
[14]	valid_0's rmse: 1.70174
[15]	valid_0's rmse: 1.69912
[16]	valid_0's rmse: 1.69655
[17]	valid_0's rmse: 1.69368
[18]	valid_0's rmse: 1.69111
[19]	valid_0's rmse: 1.68873
[20]	valid_0's rmse: 1.68614
[21]	valid_0's rmse: 1.68322
[22]	valid_0's rmse: 1.68106
[23]	valid_0's rmse: 1.67854
[24]	valid_0's rmse: 1.67562
[25]	valid_0's rmse: 1.67322
[26]	valid_0's rmse: 1.67093
[27]	valid_0's rmse: 1.66864
[28]	valid_0's rmse: 1.66628
[29]	valid_0's rmse: 1.66408
[30]	valid_0's rmse: 1.66164
[31]	valid_0's rmse: 1.65921
[32]	valid_0's rmse: 1.6569
[33]

[289]	valid_0's rmse: 1.43217
[290]	valid_0's rmse: 1.43191
[291]	valid_0's rmse: 1.43167
[292]	valid_0's rmse: 1.4315
[293]	valid_0's rmse: 1.43156
[294]	valid_0's rmse: 1.43136
[295]	valid_0's rmse: 1.43105
[296]	valid_0's rmse: 1.43092
[297]	valid_0's rmse: 1.43081
[298]	valid_0's rmse: 1.4306
[299]	valid_0's rmse: 1.43025
[300]	valid_0's rmse: 1.43022
[301]	valid_0's rmse: 1.43001
[302]	valid_0's rmse: 1.42978
[303]	valid_0's rmse: 1.42938
[304]	valid_0's rmse: 1.42922
[305]	valid_0's rmse: 1.42901
[306]	valid_0's rmse: 1.42899
[307]	valid_0's rmse: 1.4288
[308]	valid_0's rmse: 1.42851
[309]	valid_0's rmse: 1.42823
[310]	valid_0's rmse: 1.42813
[311]	valid_0's rmse: 1.42789
[312]	valid_0's rmse: 1.42782
[313]	valid_0's rmse: 1.4276
[314]	valid_0's rmse: 1.42737
[315]	valid_0's rmse: 1.42722
[316]	valid_0's rmse: 1.42682
[317]	valid_0's rmse: 1.42665
[318]	valid_0's rmse: 1.42641
[319]	valid_0's rmse: 1.4263
[320]	valid_0's rmse: 1.4263
[321]	valid_0's rmse: 1.426
[322]	valid_0's rm

In [7]:
X=train[train.columns[2:]]
y=np.log1p(train['target'])

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=203)

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'rmse'},
    'num_leaves': 50,
    'learning_rate': 0.008,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 4,
    'max_depth': -1,
    'reg_alpha': 0.3,
    'reg_lambda': 0.1,
    'min_child_weight': 10,
    'zero_as_missing': True,
    'verbose': 1
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)


scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(gbm.predict(x_train, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(gbm.predict(x_test, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]

model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        validation_data=(x_test, y_test),
        callbacks=cbs)

[1]	valid_0's rmse: 1.73905
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's rmse: 1.73622
[3]	valid_0's rmse: 1.7331
[4]	valid_0's rmse: 1.73015
[5]	valid_0's rmse: 1.72709
[6]	valid_0's rmse: 1.72404
[7]	valid_0's rmse: 1.72075
[8]	valid_0's rmse: 1.71815
[9]	valid_0's rmse: 1.71527
[10]	valid_0's rmse: 1.71258
[11]	valid_0's rmse: 1.71013
[12]	valid_0's rmse: 1.70768
[13]	valid_0's rmse: 1.70481
[14]	valid_0's rmse: 1.70174
[15]	valid_0's rmse: 1.69912
[16]	valid_0's rmse: 1.69655
[17]	valid_0's rmse: 1.69368
[18]	valid_0's rmse: 1.69111
[19]	valid_0's rmse: 1.68873
[20]	valid_0's rmse: 1.68614
[21]	valid_0's rmse: 1.68322
[22]	valid_0's rmse: 1.68106
[23]	valid_0's rmse: 1.67854
[24]	valid_0's rmse: 1.67562
[25]	valid_0's rmse: 1.67322
[26]	valid_0's rmse: 1.67093
[27]	valid_0's rmse: 1.66864
[28]	valid_0's rmse: 1.66628
[29]	valid_0's rmse: 1.66408
[30]	valid_0's rmse: 1.66164
[31]	valid_0's rmse: 1.65921
[32]	valid_0's rmse: 1.6569
[33]	valid_0's rmse: 1

[277]	valid_0's rmse: 1.43525
[278]	valid_0's rmse: 1.43531
[279]	valid_0's rmse: 1.43532
[280]	valid_0's rmse: 1.43521
[281]	valid_0's rmse: 1.4347
[282]	valid_0's rmse: 1.43434
[283]	valid_0's rmse: 1.43388
[284]	valid_0's rmse: 1.43373
[285]	valid_0's rmse: 1.43341
[286]	valid_0's rmse: 1.43306
[287]	valid_0's rmse: 1.43251
[288]	valid_0's rmse: 1.43233
[289]	valid_0's rmse: 1.43217
[290]	valid_0's rmse: 1.43191
[291]	valid_0's rmse: 1.43167
[292]	valid_0's rmse: 1.4315
[293]	valid_0's rmse: 1.43156
[294]	valid_0's rmse: 1.43136
[295]	valid_0's rmse: 1.43105
[296]	valid_0's rmse: 1.43092
[297]	valid_0's rmse: 1.43081
[298]	valid_0's rmse: 1.4306
[299]	valid_0's rmse: 1.43025
[300]	valid_0's rmse: 1.43022
[301]	valid_0's rmse: 1.43001
[302]	valid_0's rmse: 1.42978
[303]	valid_0's rmse: 1.42938
[304]	valid_0's rmse: 1.42922
[305]	valid_0's rmse: 1.42901
[306]	valid_0's rmse: 1.42899
[307]	valid_0's rmse: 1.4288
[308]	valid_0's rmse: 1.42851
[309]	valid_0's rmse: 1.42823
[310]	valid_0'

3567/3567 [==============================] - 0s 16us/step - loss: 1.3355 - val_loss: 1.3069
Epoch 24/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2802 - val_loss: 1.3278
Epoch 25/1000
3567/3567 [==============================] - 0s 14us/step - loss: 1.2677 - val_loss: 1.3140
Epoch 26/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2459 - val_loss: 1.3104
Epoch 27/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2370 - val_loss: 1.3122
Epoch 28/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2405 - val_loss: 1.2978
Epoch 29/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2502 - val_loss: 1.3135
Epoch 30/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2422 - val_loss: 1.3042
Epoch 31/1000
3567/3567 [==============================] - 0s 15us/step - loss: 1.2450 - val_loss: 1.3566
Epoch 32/1000
3567/3567 [==============================] - 0

## NN+ElNet - Actual

In [9]:
x_train=train[train.columns[2:]]
y_train=np.log1p(train['target'])

x_test=test[x_train.columns]

sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

def root_mean_squared_error(y_true, y_pred):
        return abs(K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)))

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'gamma',
    'metric': {'rmse'},
    'num_leaves': 50,
    'learning_rate': 0.008,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 4,
    'max_depth': -1,
    'reg_alpha': 0.3,
    'reg_lambda': 0.1,
    'min_child_weight': 10,
    'zero_as_missing': True,
    'verbose': 1
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000)


scal=StandardScaler()
x_train = np.c_[x_train,scal.fit_transform(gbm.predict(x_train, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]
x_test = np.c_[x_test,scal.transform(gbm.predict(x_test, num_iteration=gbm.best_iteration).flatten().reshape(-1,1)).flatten()]


model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss=root_mean_squared_error,
              optimizer=Adam(lr=0.1,decay=0.0001))


checkp = ModelCheckpoint(filepath='weights.hdf5')
lrred = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=10, cooldown=2, min_lr=0.000001)
stp = EarlyStopping(monitor='loss', min_delta=0, patience=50)
cbs = [checkp,lrred,stp]
model.fit(x_train, y_train,
        epochs=1000,
        batch_size=400,
        callbacks=cbs)

Epoch 1/1000
4459/4459 [==============================] - 0s 52us/step - loss: 14.3501
Epoch 2/1000
4459/4459 [==============================] - 0s 14us/step - loss: 8.7264
Epoch 3/1000
4459/4459 [==============================] - 0s 14us/step - loss: 7.2545
Epoch 4/1000
4459/4459 [==============================] - 0s 13us/step - loss: 6.7290
Epoch 5/1000
4459/4459 [==============================] - 0s 14us/step - loss: 5.7271
Epoch 6/1000
4459/4459 [==============================] - 0s 14us/step - loss: 5.1289
Epoch 7/1000
4459/4459 [==============================] - 0s 14us/step - loss: 4.6125
Epoch 8/1000
4459/4459 [==============================] - 0s 14us/step - loss: 4.0757
Epoch 9/1000
4459/4459 [==============================] - 0s 13us/step - loss: 3.4672
Epoch 10/1000
4459/4459 [==============================] - 0s 14us/step - loss: 2.9047
Epoch 11/1000
4459/4459 [==============================] - 0s 13us/step - loss: 2.5308
Epoch 12/1000
4459/4459 [==========================

4459/4459 [==============================] - 0s 15us/step - loss: 1.0767
Epoch 96/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0457
Epoch 97/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0524
Epoch 98/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0368
Epoch 99/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0347
Epoch 100/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0482
Epoch 101/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0522
Epoch 102/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0337
Epoch 103/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0292
Epoch 104/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0578
Epoch 105/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0409
Epoch 106/1000
4459/4459 [=========================

4459/4459 [==============================] - 0s 14us/step - loss: 0.9976
Epoch 189/1000
4459/4459 [==============================] - 0s 14us/step - loss: 0.9982
Epoch 190/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0040
Epoch 191/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0059
Epoch 192/1000
4459/4459 [==============================] - 0s 14us/step - loss: 0.9960
Epoch 193/1000
4459/4459 [==============================] - 0s 13us/step - loss: 0.9960
Epoch 194/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0019
Epoch 195/1000
4459/4459 [==============================] - 0s 13us/step - loss: 0.9971
Epoch 196/1000
4459/4459 [==============================] - 0s 14us/step - loss: 1.0054
Epoch 197/1000
4459/4459 [==============================] - 0s 13us/step - loss: 0.9961
Epoch 198/1000
4459/4459 [==============================] - 0s 13us/step - loss: 1.0113
Epoch 199/1000
4459/4459 [=====================

In [10]:
predictions=pd.DataFrame({'ID':test['ID'],'target':np.expm1(model.predict(x_test, verbose=1).flatten())})
print(predictions.head())
predictions.to_csv('pred_nn_lgbm.csv',index=False)

49342/49342 [==============================] - 1s 14us/step
          ID      target
0  000137c73  1728998.25
1  00021489f  2572478.50
2  0004d7953  3053373.50
3  00056a333  3818850.75
4  00056d8eb  2572478.50
